# Práctica sobre algoritmos genéticos

<img src="imgs/tren.jpg">


Un área ferroviaria de carga/descarga con una única vía de entrada y otra salida se compone de tres muelles de carga/descarga: Op1, Op2 y Op3, correspondientes a contenedores, carbón y gas. Por tanto, cada tren que llega se dirige a un muelle en función de su carga. Un tren tarda en cargar o descargar un tiempo proporcional al número de vagones que arrastra. Cada día llegan secuencialmente n trenes. Si los trenes son de cargas distintas, pueden entrar en paralelo a los muelles. Cuando dos trenes con el mismo tipo de carga se encuentran seguidos, el segundo debe esperar por el primero, así como todos los demás que se encuentren por detrás.


Se nos plantea resolver, mediante un algoritmo genético, el problema de la ordenación en la entrada de los trenes para minimizar el tiempo de paso del conjunto de trenes.


<img src="imgs/docks.jpg" width="70%">

In [1]:
import random
import numpy
from deap import base, creator, tools, algorithms

## Trenes

En el siguiente código, <code>random_trains_generation</code> genera los trenes que llegarán en un día. Cada ordenación diferente de este conjunto corresponderá con un **individuo** en nuestro eaquema de algoritmo genético.

In [2]:
import random
# from deap import base, creator, tools

class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)
    
    def __repr__(self):
        return f"Train({self.wagons}, '{self.op}', {self.licence_plate})"

def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op, i))

    return train_list


In [3]:
trains = random_trains_generation(25)

print(trains[7])
trains[7]

Número de vagones:29
Muelle de operaciones:op1
Matrícula:7


Train(29, 'op1', 7)

# Solución

In [4]:
from collections import defaultdict

def trains_cost(indices, trains):
    loaders = defaultdict(lambda: 0)
    time = 0
    for i in indices:
        train = trains[i]
        time = max(time, loaders[train.op])
        loaders[train.op] = time + train.wagons
    return max(*loaders.values()),


In [5]:
def lower_bound(trains):
    loaders = defaultdict(lambda: 0)
    for train in trains:
        loaders[train.op] += train.wagons
    return max(*loaders.values())

# Usando DEAP

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [7]:
def shuffle_trains(size):
    ind = creator.Individual(range(size))
    random.shuffle(ind)
    return ind

In [15]:
TRAIN_LEN = 25
all_trains = random_trains_generation(TRAIN_LEN)

In [29]:
import numpy as np


toolbox = base.Toolbox()
toolbox.register("individual", shuffle_trains, TRAIN_LEN)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", trains_cost, trains=all_trains)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=3000)

In [23]:
lb = lower_bound(all_trains)
lb

217

In [30]:
hof = tools.HallOfFame(4)

stats = tools.Statistics(lambda indiv: indiv.fitness.values[0])
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("innefic", lambda f: np.min(f) / lb)
stats.register("wasted_time", lambda f: np.min(f) - lb)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)


gen	nevals	avg    	min	max	innefic	wasted_time
0  	3000  	286.645	221	397	1.01843	4          
1  	1830  	267.786	217	385	1      	0          
2  	1763  	256.637	217	344	1      	0          
3  	1771  	249.982	217	352	1      	0          
4  	1835  	245.707	217	343	1      	0          
5  	1838  	242.489	217	351	1      	0          
6  	1788  	239.589	217	316	1      	0          
7  	1801  	237.817	217	317	1      	0          
8  	1771  	235.953	217	325	1      	0          
9  	1811  	234.592	217	347	1      	0          
10 	1825  	234.395	217	325	1      	0          
11 	1733  	232.371	217	341	1      	0          
12 	1814  	232.196	217	320	1      	0          
13 	1769  	231.469	217	311	1      	0          
14 	1830  	231.833	217	308	1      	0          
15 	1780  	231.611	217	326	1      	0          
16 	1812  	230.993	217	313	1      	0          
17 	1784  	231.29 	217	317	1      	0          
18 	1806  	231.322	217	331	1      	0          
19 	1759  	230.543	217	330	1      	0          
20 	1779  	23

In [14]:
for i in range(4):
	print(hof[i], hof[i].fitness.values)

[5, 2, 3, 12, 6, 14, 11, 17, 13, 19, 15, 23, 4, 7, 20, 18, 22, 10, 16, 0, 1, 21, 8, 24, 9] (229.0,)
[23, 20, 2, 5, 18, 22, 6, 11, 13, 9, 3, 10, 15, 19, 7, 0, 17, 4, 14, 1, 8, 16, 21, 12, 24] (229.0,)
[5, 2, 3, 12, 6, 21, 11, 17, 13, 19, 15, 23, 20, 7, 4, 18, 0, 10, 16, 22, 1, 14, 8, 24, 9] (229.0,)
[5, 2, 3, 12, 6, 14, 11, 17, 13, 19, 15, 23, 20, 7, 4, 18, 0, 10, 16, 22, 1, 21, 8, 24, 9] (229.0,)
